# Some bugs I found in PyMEOS

In [1]:
import pandas as pd
from functools import reduce
from pymeos import *
from pysparkmeos.UDT.MeosDatatype import *
from pysparkmeos.partitions.grid.grid_partitioner import *

from pymeos_cffi import *

pymeos_initialize("UTC")

Assume we have one TGeogSequence like this:

In [2]:
tseq = TGeogPointSeq(string='[POINT(-81.5 28.73)@2022-06-27 00:00:10+00, POINT(-81.5 28.72)@2022-06-27 00:00:20+00, POINT(-81.48 28.7)@2022-06-27 00:00:40+00, POINT(-81.48 28.69)@2022-06-27 00:00:50+00, POINT(30.45 -70.02)@2022-06-27 00:00:55+00)')
print(tseq.srid())

4326


and a STBox (that in my project I use to represent the boundaries of a table) like this:

<i>Note how upon creation the STBox srid is incorrectly set to 0 even if we specify we want it to be geodetic.</i>

In [3]:
bounds = STBox("STBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])")
print(bounds.srid())

# In this case the srid is changed but the string representation is incorrect according to the MobilityDB manual.
# String representation should be: SRID=4326;GEODSTBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])
# It adds the 'SRID=4326;' to the string representation, but omits the 'GEOD'.
print(bounds.set_srid(4326).srid(), bounds.set_srid(4326).__str__())

boundsgeod = STBox(
    "STBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])",
    geodetic=True
)

# If we ask the STBox to be geodetic from the beginning, the srid should be 4326.
print(boundsgeod.srid())  # This outputs 0
print(boundsgeod.set_srid(4326).srid())
# This outputs 4326 but the string representation (and the inner) is wrong. 

0
4326 SRID=4326;STBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])
0
4326


In [4]:
bounds = STBox("STBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])")
bounds = bounds.set_srid(4326)
# This string representation, even with the 4326 but missing the GEOD, causes issues.
print(bounds.__str__())

# For now, the way I am working around this error is by doing:
bounds = STBox(f"SRID=4326;GEOD{bounds.__str__().replace('SRID=4326;', '')}", geodetic=True)
print(bounds) # This is correct after the tweak

SRID=4326;STBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])
SRID=4326;GEODSTBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])


In [5]:
# Doing the opposite (from geodetic to planar)
bounds = STBox("SRID=4326;GEODSTBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00])")
print(bounds.srid())


bounds = bounds.set_srid(0)
print(bounds, bounds.srid()) # This shouldn't contain the 'GEOD', so we have to do another tweak

bounds = STBox(f"SRID=0;{bounds.__str__().replace('GEOD', '')}", geodetic=False)
print(bounds, bounds.srid()) # This looks correct now

4326
GEODSTBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00]) 0
STBOX XT(((-177.02969360351562,-46.421356201171875),(177.816650390625,70.29727935791016)),[2022-06-27 00:00:00+00, 2022-06-27 00:15:00+00]) 0


In [6]:
# Tile mtehod only allows planar coordinates
grid = bounds.tile(size=150, duration='10m')
print(grid)

[STBox(STBOX XT(((-300,-150),(-150,0)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((-150,-150),(0,0)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((0,-150),(150,0)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((150,-150),(300,0)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((-300,0),(-150,150)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((-150,0),(0,150)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((0,0),(150,150)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((150,0),(300,150)),[2022-06-27 00:00:00+00, 2022-06-27 00:10:00+00))), STBox(STBOX XT(((-300,-150),(-150,0)),[2022-06-27 00:10:00+00, 2022-06-27 00:20:00+00))), STBox(STBOX XT(((-150,-150),(0,0)),[2022-06-27 00:10:00+00, 2022-06-27 00:20:00+00))), STBox(STBOX XT(((0,-150),(150,0)),[2022-06-27 00:10:00+00, 2022-06-27 00:20:00+00))), STBox(STBOX XT(((150,-150),(300,0)),[2

In [14]:
# Now if we try the .at() method

tseq = TGeomPointSeq(string='SRID=0;[POINT(-81.5 28.73)@2022-06-27 00:00:10+00, POINT(-81.5 28.72)@2022-06-27 00:00:20+00, POINT(-81.48 28.7)@2022-06-27 00:00:40+00, POINT(-81.48 28.69)@2022-06-27 00:00:50+00, POINT(30.45 -70.02)@2022-06-27 00:00:55+00)')
tseq = tseq.set_srid(0)

print(tseq)

for key, tile in enumerate(grid):
    print(tile.srid(), tseq.srid())
    
    if tseq.at(tile) is not None:
        print("Attempting: ", key, tseq.at(tile))

[POINT(-81.5 28.73)@2022-06-27 00:00:10+00, POINT(-81.5 28.72)@2022-06-27 00:00:20+00, POINT(-81.48 28.7)@2022-06-27 00:00:40+00, POINT(-81.48 28.69)@2022-06-27 00:00:50+00, POINT(30.45 -70.02)@2022-06-27 00:00:55+00)
0 0
0 0
Attempting:  1 {[POINT(-48.947635044 0.000017468000003)@2022-06-27 00:00:51.453246+00, POINT(-0.000019236 -43.166418308000004)@2022-06-27 00:00:53.639774+00)}
0 0
Attempting:  2 {[POINT(-0.000019236 -43.166418308000004)@2022-06-27 00:00:53.639774+00, POINT(30.45 -70.02)@2022-06-27 00:00:55+00)}
0 0
0 0
0 0
Attempting:  5 {[POINT(-81.5 28.73)@2022-06-27 00:00:10+00, POINT(-81.5 28.72)@2022-06-27 00:00:20+00, POINT(-81.48 28.7)@2022-06-27 00:00:40+00, POINT(-81.48 28.69)@2022-06-27 00:00:50+00, POINT(-48.947635044 0.000017468000003)@2022-06-27 00:00:51.453246+00)}
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
